In [36]:
from __future__ import annotations
from dataclasses import dataclass, field, fields
import functools


def rsetattr(obj, attr, val):
    pre, _, post = attr.rpartition('.')
    return setattr(rgetattr(obj, pre) if pre else obj, post, val)

def rgetattr(obj, attr, *args):
    def _getattr(obj, attr):
        return getattr(obj, attr, *args)
    return functools.reduce(_getattr, [obj] + attr.split('.'))


@dataclass
class BgSub:
    sigma: float = field(default_factory=float)
    size: int = field(default_factory=int)
    overwrite: bool = field(default_factory=bool)
    
@dataclass
class ChanShift:
    reg_channel: str = field(default_factory=str)
    reg_mtd: str = field(default_factory=str)
    overwrite: bool = field(default_factory=bool)
    
@dataclass
class Register:
    reg_channel: str = field(default_factory=str)
    reg_mtd: str = field(default_factory=str)
    reg_ref: str = field(default_factory=str)
    overwrite: bool = field(default_factory=bool)
    
@dataclass
class Blur:
    kernel: tuple[int] = field(default_factory=tuple)
    sigma: int = field(default_factory=int)
    img_fold_src: str = field(default_factory=str)
    overwrite: bool = field(default_factory=bool)

def unpack_settings(input_settings: dict)-> Settings:
    settings = Settings()
    if input_settings['run_bg_sub']:
        settings.bg_sub = BgSub(**input_settings['bg_sub'])
    if input_settings['run_chan_shift']:
        settings.chan_shift = ChanShift(**input_settings['chan_shift'])
    if input_settings['run_register']:
        settings.register = Register(**input_settings['register'])
    if input_settings['run_blur']:
        settings.blur = Blur(**input_settings['blur'])
    return settings


@dataclass
class Settings:
    settings: dict
    bg_sub: dict = field(init=False)
    chan_shift: dict = field(init=False)
    register: dict = field(init=False)
    blur: dict = field(init=False)
    
    def __post_init__(self)-> None:
        if self.settings['run_bg_sub']:
            self.bg_sub = self.settings['bg_sub']
        if self.settings['run_chan_shift']:
            self.chan_shift = self.settings['chan_shift']
        if self.settings['run_register']:
            self.register = self.settings['register']
        if self.settings['run_blur']:
            self.blur = self.settings['blur']
        self.update_overwrite()
        
    def update_overwrite(self)-> None:
        active_branches = [f.name for f in fields(self) if hasattr(self,f.name) and f.name != 'settings']
        current_overwrite = [getattr(self,f)['overwrite'] for f in active_branches]

        # Get the new overwrite list, if the previous is true then change the next to true, else keep the same
        new_overwrite = []; is_False = True
        for i in range(len(current_overwrite)):
            if current_overwrite[i] == False and is_False:
                new_overwrite.append(current_overwrite[i])
            elif current_overwrite[i] == True and is_False:
                new_overwrite.append(current_overwrite[i])
                is_False = False
            elif not is_False:
                new_overwrite.append(True)# Update the overwrite attribute
        
        # Update the overwrite attribute
        for i,branch in enumerate(active_branches):
            temp_dict = getattr(self,branch)
            temp_dict['overwrite'] = new_overwrite[i]
            setattr(self,branch,temp_dict)


settings = settings = {
    "run_bg_sub": True,
    "run_chan_shift": True,
    "run_register": True,
    "run_blur": True,
    
    "bg_sub": {"sigma": 0.0,"size": 7,"overwrite": True},
    "chan_shift": {"reg_channel": "C1","reg_mtd": "rigid","overwrite": False},
    "register": {"reg_channel": "C1","reg_mtd": "rigid","reg_ref": "C1","overwrite": False},
    "blur": {"kernel": (5,5),"sigma": 0,"img_fold_src": "","overwrite": True}}

s = Settings(settings)
print(f"{s.chan_shift['overwrite'] = }")
s.update_overwrite()
print(f"{s.chan_shift['overwrite'] = }")

s.chan_shift['overwrite'] = False
s.chan_shift['overwrite'] = True
